In [1]:
from contrans import contrans
import numpy as np
import pandas as pd
import dotenv
import os
import json
import requests 
import psycopg
import sqlite3
from sqlalchemy import create_engine    
dotenv.load_dotenv()
congresskey = os.getenv('congresskey')
postgrespassword = os.getenv('POSTGRES_PASSWORD')

In [2]:
ct = contrans()

In [3]:
#Creates a new, empty contrans database
dbserver, engine = ct.connect_to_postgres(ct.POSTGRES_PASSWORD, create_contrans=True)

In [4]:
members = ct.get_bioguideIDs() # members from Congress API
members = ct.make_cand_table(members) # joining the contributions ID with the Congress API data
terms, members = ct.terms_df(members) # separates the terms (non-atomic) data from members
ideology = ct.get_ideology() # gets the Ideology data from voteview.com
ct.make_members_df(members, ideology, engine) # joins members and ideology and uploads to postgres DB

In [5]:
ct.make_terms_df(terms, engine)

In [6]:
votes = ct.get_votes()
ct.make_votes_df(votes, engine)

In [7]:
# SQL queries
myquery = '''
SELECT *
FROM members
WHERE state = 'VA'
'''
pd.read_sql_query(myquery, con=engine)

,bioguideid,district,name,partyname,state,updatedate,url,depiction.attribution,depiction.imageurl,partyletter,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,K000399,02,"Kiggans, Jennifer A.",Republican,VA,2024-08-05T15:15:39Z,https://api.congress.gov/v3/member/K000399?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66b0ce45b0...,R,...,None,0.258,0.431,-104.64907,0.90471,1045.0,34.0,None,0.258,0.430
1,G000595,05,"Good, Bob",Republican,VA,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/G000595?for...,Image courtesy of the Member,https://www.congress.gov/img/member/g000595_20...,R,...,None,0.800,-0.600,-77.56145,0.92833,1043.0,34.0,None,0.769,-0.639
2,W000825,10,"Wexton, Jennifer",Democratic,VA,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/W000825?for...,Image courtesy of the Member,https://www.congress.gov/img/member/w000825_20...,D,...,None,-0.384,0.302,-42.16114,0.95458,907.0,14.0,None,-0.427,0.174
3,S001209,07,"Spanberger, Abigail Davis",Democratic,VA,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/S001209?for...,Image courtesy of the Member,https://www.congress.gov/img/member/s001209_20...,D,...,None,-0.197,0.313,-105.49853,0.90510,1058.0,35.0,None,-0.262,0.291
4,C001118,06,"Cline, Ben",Republican,VA,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/C001118?for...,Image courtesy of the Member,https://www.congress.gov/img/member/c001118_20...,R,...,None,0.715,-0.212,-79.77979,0.92783,1065.0,30.0,None,0.709,-0.259
5,B001292,08,"Beyer, Donald S.",Democratic,VA,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/B001292?for...,Image courtesy of the Member,https://www.congress.gov/img/member/b001292_20...,D,...,None,-0.390,-0.100,-74.64359,0.93100,1044.0,31.0,None,-0.403,-0.216
6,G000568,09,"Griffith, H. Morgan",Republican,VA,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/G000568?for...,Image courtesy of the Member,https://www.congress.gov/img/member/g000568_20...,R,...,None,0.514,-0.350,-242.40046,0.78243,988.0,109.0,None,0.445,-0.278
7,C001078,11,"Connolly, Gerald E.",Democratic,VA,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/C001078?for...,Image courtesy of the Member,https://www.congress.gov/img/member/c001078_20...,D,...,None,-0.308,-0.045,-78.78733,0.92738,1045.0,32.0,None,-0.348,-0.208
8,M001227,04,"McClellan, Jennifer L.",Democratic,VA,2024-06-08T18:40:21Z,https://api.congress.gov/v3/member/M001227?for...,Image courtesy of the Member,https://www.congress.gov/img/member/m001227_20...,D,...,None,-0.551,0.035,-37.82573,0.96023,932.0,16.0,None,-0.545,0.038
9,W000804,01,"Wittman, Robert J.",Republican,VA,2024-06-08T18:40:21Z,https://api.congress.gov/v3/member/W000804?for...,Image courtesy of the Member,https://www.congress.gov/img/member/w000804_20...,R,...,None,0.448,0.019,-131.40576,0.88320,1058.0,61.0,None,0.508,0.189


In [11]:
myquery = '''
SELECT *
FROM votes
'''
data = pd.read_sql_query(myquery, con=engine)
print(ct.dbml_helper(data))

  congress     int
   chamber varchar
rollnumber     int
     icpsr     int
 cast_code     int
      prob   float
